# Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Prepare inputs and targets

In [2]:
fb_sentiment = pd.read_csv("sentiment_time_series/facebook_sentiment_time_series.csv")
fb_sentiment.head()

,date,binary_mean,binary_squared_mean,binary_sqrt_mean,prob_mean,prob_squared_mean,prob_sqrt_mean
0,1/1/2020,0.0,0.0,0.0,0.0,0.0,0.0
1,1/2/2020,0.0,0.0,0.0,0.0,0.0,0.0
2,1/3/2020,0.0,0.0,0.0,0.0,0.0,0.0
3,1/4/2020,0.0,0.0,0.0,0.0,0.0,0.0
4,1/5/2020,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
fb_lstm = pd.read_csv("FB_LSTM_predictions.csv")
fb_lstm.head()

,Unnamed: 0,Date,Pred,Actual
0,1916,1/2/2020,208.330950,209.779999
1,1917,1/3/2020,212.493704,208.669998
2,1918,1/6/2020,211.364467,212.600006
3,1919,1/7/2020,215.023133,213.059998
4,1920,1/8/2020,215.540427,215.220001


In [4]:
merged = pd.merge(fb_sentiment, fb_lstm, left_on='date', right_on='Date')

In [5]:
merged_changed = merged[['date', 'prob_sqrt_mean', 'Pred', 'Actual']]
merged_changed = merged_changed.dropna()

In [6]:
inputs_1 = merged_changed.prob_sqrt_mean.values
inputs_1.shape

(45,)

In [7]:
inputs_2 = merged_changed.Pred.values
inputs_2.shape

(45,)

In [8]:
inputs = np.stack((inputs_1, inputs_2), axis=1)
inputs.shape

(45, 2)

In [9]:
targets = merged_changed.iloc[:, 3].values

# Fit a model

In [10]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
inputs = sc.fit_transform(inputs)

inputs = sm.add_constant(inputs)
model = sm.OLS(targets, inputs)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                     1182.
Date:                Sat, 13 Feb 2021   Prob (F-statistic):           1.21e-37
Time:                        17:52:21   Log-Likelihood:                -130.76
No. Observations:                  45   AIC:                             267.5
Df Residuals:                      42   BIC:                             272.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        224.5582      0.682    329.042      0.0

# Save results

In [11]:
predictions = results.predict()

In [12]:
fb_stock_prediction = pd.DataFrame().assign(Date=merged_changed['date'], Predictions=predictions, 
                                                Actual=merged_changed['Actual'])
fb_stock_prediction.to_csv("fb_stock_prediction_reddit.csv")